##### https://www.kaggle.com/competitions/nlp-getting-started
##### Addison Howard, devrishi, Phil Culliton, Yufeng Guo. (2019). Natural Language Processing with Disaster Tweets
###### the following notebook is very useful:
###### https://www.kaggle.com/code/chayan8/sentiment-analysis-using-bert-pytorch

In [392]:
import numpy as np 
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import TensorDataset

In [393]:
### Dataset: Disaster: target==1. If no_Disaster: traget==0
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [394]:
tmp = df_train.sample(frac=1).reset_index(drop=True)
print(len(df_train))

7613


In [395]:
# train val spilit from train-data
df_train = tmp[:6000]
df_val   = tmp[6000:]

print(df_train.shape)
print(df_val.shape)


(6000, 5)
(1613, 5)


##### https://huggingface.co/docs/transformers/model_doc/bert
##### https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification.forward.example

### Ways to use BERT
##### 1.Use BERT pre-trained model
##### 2.Finetune BERT base for classification task

In [396]:
##Example-1
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
# text = "how are you doing"
# tokenized_text=tokenizer.tokenize(text)
# print('tokenized_test:')
# print(tokenized_text)
# print()

# encoded_text = tokenizer(text, padding=True,truncation=True, max_length=10, return_tensors='pt')
# print('encoded_text:')
# print(encoded_text)
# print()

# decoded_text=tokenizer.decode(encoded_text["input_ids"][0],skip_special_tokens=False)
# print('decoded_text:')
# print(decoded_text)
# print()

# #embedded text in vector format
# text_embeddings = model(**encoded_text)
# print('embedded vector:')
# print(text_embeddings[0])
# print(text_embeddings[0].shape)

In [397]:
##Example-2
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
# #padded/trimmed to custom token length
# text = ["how are you doing",\
#         "hello sir",\
#         "is there any whay to reach the destination without checking the google map and etc etc"]

# encoded_text = tokenizer(text, padding=True,truncation=True, max_length=10, return_tensors='pt')
# print('encoded_text:')
# print(encoded_text['input_ids'])
# print(encoded_text['input_ids'].shape)
# print()

# decoded_text=tokenizer.decode(encoded_text["input_ids"][0],skip_special_tokens=False)
# print('decoded_text:')
# print(tokenizer.decode(encoded_text["input_ids"][0],skip_special_tokens=False))
# print(tokenizer.decode(encoded_text["input_ids"][1],skip_special_tokens=False))
# print(tokenizer.decode(encoded_text["input_ids"][2],skip_special_tokens=False))

#### Using Pre-trained Model
##### https://huggingface.co/textattack/bert-base-uncased-yelp-polarity

In [398]:
'''
------------------------------------------
Large Yelp Review Dataset. This is a dataset for binary sentiment classification
The Yelp reviews polarity dataset is constructed by considering stars 1 and 2 negative, and 3 and 4 positive.
Negative polarity: class 1, positive polarity:class 2.
predicted_class_id: 0 means Negative
predicted_class_id: 1 means Positive
------------------------------------------
But for our disater tweets dataset. 
We will still try to use this pre-trained model to see the prediction accuracy
predicted_class_id: 0 means Positive (No Disaster)
predicted_class_id: 1 means Negative (Disaster)
Hence use: logits*torch.tensor([-1])
------------------------------------------
'''
#Example case for seq. classification
from transformers import AutoTokenizer, BertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

### Example for BertForSequenceClassification
inputs = tokenizer("Hello, my dog is dangerous", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = (logits*torch.tensor([-1])).argmax().item()
model.config.id2label[predicted_class_id]

labels = torch.tensor([0]) #specify label
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

print('logits:',logits)
print('predicted_class_id:',predicted_class_id)
print('id2label:',model.config.id2label[predicted_class_id])
print('loss:',loss)

# '''
# Output:
# logits: tensor([[ 4.3746, -3.9057]])
# predicted_class_id: 0
# id2label: LABEL_0
# loss: tensor(0.0003, grad_fn=<NllLossBackward0>)
# '''

logits: tensor([[ 0.6456, -0.8177]])
predicted_class_id: 1
id2label: LABEL_1
loss: tensor(0.2082, grad_fn=<NllLossBackward0>)


In [399]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train.text.values.tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=100,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_val.text.values.tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    max_length=100,
    padding=True,
    return_tensors='pt'
)

encoded_input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.target.values)

encoded_input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val.target.values)

In [400]:
# ### calculate the prediction accuracy for train-set 
# with torch.no_grad():
#     logits = model(**encoded_data_train).logits
# predicted_class_id=torch.argmax(logits*torch.tensor([-1]),dim=1)

# # Calculate the number of correct predictions
# correct_predictions = (predicted_class_id == labels_train).sum().item()
# # Calculate accuracy
# accuracy = correct_predictions / labels_train.size(0)  # total number of samples
# print(f'Accuracy: {accuracy * 100:.2f}%')

In [401]:
# ### calculate the prediction accuracy for val-set 
# with torch.no_grad():
#     logits = model(**encoded_data_val).logits
# predicted_class_id=torch.argmax(logits*torch.tensor([-1]),dim=1)

# # Calculate the number of correct predictions
# correct_predictions = (predicted_class_id == labels_val).sum().item()
# # Calculate accuracy
# accuracy = correct_predictions / labels_val.size(0)  # total number of samples
# print(f'Accuracy: {accuracy * 100:.2f}%')

####  Finetune BERT for Classification

In [402]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import f1_score

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = 2,
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [403]:
dataset_train = TensorDataset(encoded_input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(encoded_input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [404]:
dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=32
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=len(labels_val)
)

In [405]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

/home/vino/anaconda3/envs/pytorch23/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [411]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [407]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [408]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [409]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [413]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [414]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
       
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

##save model
#torch.save(model.state_dict(), './data/bert_disaster_tweets_finetuned_weights.pth')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/188 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.3494734355109803


  0%|          | 0/1 [00:00<?, ?it/s]

Validation loss: 0.3902204632759094
F1 Score (weighted): 0.8318329192114048


In [415]:
### calculate the prediction accuracy for val-set 
with torch.no_grad():
    logits = model(**encoded_data_val).logits
predicted_class_id=torch.argmax(logits,dim=1)

# Calculate the number of correct predictions
correct_predictions = (predicted_class_id == labels_val).sum().item()
# Calculate accuracy
accuracy = correct_predictions / labels_val.size(0)  # total number of samples
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 83.26%
